# Treinamento e Validação da IA

Download das bibliotecas, configuração do ambiente e import das bibliotecas e ferramentas que serão usadas
Obs: Acesse Runtime e selecione a opção da GPU, para utilizar CUDA durante o treinamento da IA

In [1]:
!pip install transformers datasets torch seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

In [2]:
!pip uninstall -y pyarrow requests fsspec
!pip install pyarrow==14.0.1 requests==2.31.0 fsspec==2024.6.1
!pip install cudf-cu12 gcsfs google-colab ibis-framework

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0
Found existing installation: requests 2.32.3
Uninstalling requests-2.32.3:
  Successfully uninstalled requests-2.32.3
Found existing installation: fsspec 2024.5.0
Uninstalling fsspec-2024.5.0:
  Successfully uninstalled fsspec-2024.5.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires fsspec[http]<=2024.5.0,>=2023.1.0, but you have fsspec 2024.6.1 which is incompatible.
datasets 2.20.0 requires pyarrow>=15.0.0, but you have pyarrow 14.0.1 which is incompatible.
datasets 2.20.0 requires reques

In [19]:
import json
import os
import pandas as pd
import pyarrow.parquet as pq
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import classification_report
from seqeval.metrics import classification_report, f1_score
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AdamW, get_scheduler
from transformers import BertTokenizer, BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import DataCollatorForTokenClassification

## Carregar Dataset

Carregar dataset (presente nos arquivos **news_data.parquet** e **annot.parquet**. Estes arquivos são obtidos após a conversão dos dados dos arquivos json para parquet)

In [24]:
news_dataset = Dataset.from_parquet('news_data.parquet')
annotated_dataset = Dataset.from_parquet('annot.parquet')

print("Primeiras 5 linhas do dataset de notícias:")
print(news_dataset[:5])
print("Primeiras 5 linhas do dataset anotado:")
print(annotated_dataset[:5])


Primeiras 5 linhas do dataset de notícias:
{'title': ['Petrobras: produção de óleo e gás sobe 2,4% no 2º tri, a 2,69 mi de barris por dia', 'Lotofácil: prêmio de R$ 4 milhões é sorteado nesta segunda; veja dezenas', 'Quina sorteia prêmio de R$ 4,8 milhões nesta segunda; veja números', 'Olimpíadas: confira a programação desta terça-feira', 'Líder da oposição diz que González obteve 70% dos votos na Venezuela'], 'url': ['https://www.infomoney.com.br/mercados/petrobras-petr4-producao-de-petroleo-gas-segundo-trimestre-2024/', 'https://www.infomoney.com.br/consumo/lotofacil-premio-de-r-4-milhoes-e-sorteado-nesta-segunda-veja-dezenas/', 'https://www.infomoney.com.br/consumo/quina-sorteia-premio-de-r-48-milhoes-nesta-segunda-veja-numeros/', 'https://www.infomoney.com.br/consumo/olimpiadas-confira-a-programacao-desta-terca-feira/', 'https://www.infomoney.com.br/mundo/lider-da-oposicao-diz-que-gonzalez-obteve-70-dos-votos-na-venezuela/'], 'date': ['2024-07-29T23:50:25+00:00', '2024-07-29T23:22:

In [25]:
print("Estrutura do dataset de notícias:")
print(news_dataset)
print("Estrutura do dataset anotado:")
print(annotated_dataset)


Estrutura do dataset de notícias:
Dataset({
    features: ['title', 'url', 'date', 'content'],
    num_rows: 120
})
Estrutura do dataset anotado:
Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 71
})


## Tratamento e preparação dos dados para o treinamento

Primeiro, garantimos a integridade dos rótulos (ner_tags) para que estejam configurados em string. Garantimos anteriormente a integridade para o uso das ferramentas da bilblioteca dataset da HuggingFace, pois ela trabalha com arquivos no formato da Apache Arrow, e por isso estamos utilizando dados em parquet.
Mapeamos os rótulos de string para IDs numéricos e montamos um Tokenizer e colocamos o acesso ao modelo BERT large, disponibilizado na HuggingFace. Usamos o BERTimbau por ele foi treinado para ser utilizado em problemas de Token Classification em português.
Após isso, alinhamos as labels e tokenizamos o dataset do BERT.

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

def ensure_str_labels(dataset):
    def to_str_labels(batch):
        batch['ner_tags'] = [[str(label) for label in labels] for labels in batch['ner_tags']]
        return batch
    return dataset.map(to_str_labels, batched=True)

annotated_dataset = ensure_str_labels(annotated_dataset)

label2id = {label: i for i, label in enumerate(set(tag for tags in annotated_dataset['ner_tags'] for tag in tags))}
id2label = {i: label for label, i in label2id.items()}

bert_model_name = 'neuralmind/bert-large-portuguese-cased'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForTokenClassification.from_pretrained(bert_model_name, num_labels=len(label2id))

def tokenize_and_align_labels(examples):
    tokenized_inputs = bert_tokenizer(
        examples['tokens'],
        truncation=True,
        padding='max_length',
        max_length=512,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

tokenized_datasets_bert = annotated_dataset.map(tokenize_and_align_labels, batched=True)

print("Primeiras 5 linhas do dataset tokenizado:")
print(tokenized_datasets_bert[:5])


Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Primeiras 5 linhas do dataset tokenizado:
{'tokens': [['Publicidade', 'A', 'Petrobras', '(', 'PETR3', ';', 'PETR4', ')', 'fechou', 'o', 'segundo', 'trimestre', 'do', 'ano', 'com', 'produção', 'média', 'de', '2,69', 'milhões', 'de', 'barris', 'diários', '(', 'boed', ')', 'de', 'óleo', 'equivalente', '(', 'petróleo', 'e', 'gás', 'natural', ')', ',', 'alta', 'de', '2,4', '%', 'na', 'comparação', 'com', 'o', 'mesmo', 'período', 'de', '2023.', 'As', 'informações', 'constam', 'no', 'relatório', 'de', 'produção', 'da', 'companhia', ',', 'divulgado', 'nesta', 'segunda-feira', ',', '29.', 'Em', 'relação', 'ao', 'primeiro', 'trimestre', 'de', '2024', ',', 'período', 'de', 'janeiro', 'a', 'março', ',', 'houve', 'queda', 'de', '2,8', '%', 'no', 'segundo', 'trimestre.', 'Este', 'é', 'o', 'primeiro', 'resultado', 'operacional', 'da', 'nova', 'gestão', 'da', 'estatal', ',', 'de', 'Magda', 'Chambriard', ',', 'que', 'assumiu', 'a', 'posição', 'em', '24', 'de', 'maio.', 'Baixe', 'uma', 'lista', 'de', '1

## Treinamento e Avaliação do modelo

Aqui realizamos o treinamento do nosso modelo e efetuamos a avaliação, para verificar se tudo correu da forma esperada. Primeiro, preparamos os dados para o DataLoader. Configuramos o tamanho do batch (coloquei em 4 porque antes ocorreram problemas com a GPU). Após isso, montamos o DataLoader para treinamento e avaliação, efetuamos as configurações, como o número de épocas (deixei em 10 para ele conseguir ter contato com os outros rótulos que não fossem NULL (O) ), e realizamos o treinamento. Após o treinamento, avaliamos a precisão do modelo, usamos f1-score para ver como foi a performance, assim como a sua precisão, e salvamos o modelo e o tokenizer em uma pasta separada, pois os usaremos para rotular o restante do dataset.

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(x['input_ids']) for x in batch])
    attention_mask = torch.stack([torch.tensor(x['attention_mask']) for x in batch])
    labels = torch.stack([torch.tensor(x['labels']) for x in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

batch_size = 8

train_dataloader = DataLoader(tokenized_datasets_bert, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)
eval_dataloader = DataLoader(tokenized_datasets_bert, batch_size=batch_size, collate_fn=collate_fn)

optimizer = AdamW(bert_model.parameters(), lr=2e-5)
num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

def train(model, dataloader, optimizer, lr_scheduler):
    model.train()
    for batch in tqdm(dataloader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        del batch, outputs, loss
        torch.cuda.empty_cache()

def evaluate(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    for batch in tqdm(dataloader, desc="Evaluation"):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch['labels']

        all_predictions.extend(predictions.cpu().numpy().tolist())
        all_labels.extend(labels.cpu().numpy().tolist())

        del batch, outputs, predictions, labels
        torch.cuda.empty_cache()

    return all_predictions, all_labels

bert_model.to(device)

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(bert_model, train_dataloader, optimizer, lr_scheduler)

predictions, labels = evaluate(bert_model, eval_dataloader)

model_save_path = './saved_model/model'
bert_model.save_pretrained(model_save_path)

print(f"Modelo salvo")

true_predictions = [
    [id2label[pred] for pred, label in zip(prediction, label) if label != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [id2label[label] for pred, label in zip(prediction, label) if label != -100]
    for prediction, label in zip(predictions, labels)
]

results = classification_report(true_labels, true_predictions, output_dict=True)
print("Evaluation Results:", results)


Using device: cuda
Epoch 1/10


Training:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 2/10


Training:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 3/10


...

Training:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 10/10


Training:   0%|          | 0/9 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

Modelo salvo
Evaluation Results: {'acao': {'precision': 0.9428571428571428, 'recall': 0.9428571428571428, 'f1-score': 0.9428571428571428, 'support': 35}, 'banco': {'precision': 0.5, 'recall': 0.44680851063829785, 'f1-score': 0.47191011235955055, 'support': 47}, 'cidade': {'precision': 0.5692307692307692, 'recall': 0.8043478260869565, 'f1-score': 0.6666666666666667, 'support': 46}, 'empresa': {'precision': 0.7584541062801933, 'recall': 0.9127906976744186, 'f1-score': 0.8284960422163589, 'support': 172}, 'empresario': {'precision': 0.5833333333333334, 'recall': 0.2413793103448276, 'f1-score': 0.34146341463414637, 'support': 29}, 'estado': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 15}, 'organizacao': {'precision': 0.6601941747572816, 'recall': 0.918918918918919, 'f1-score': 0.7683615819209041, 'support': 74}, 'outras_pessoas': {'precision': 0.6909090909090909, 'recall': 0.76, 'f1-score': 0.7238095238095238, 'support': 50}, 'pais': {'precision': 0.8347826086956521, 'rec

In [20]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
tokenizer_save_path = './saved_model/tokenizer'
tokenizer.save_pretrained(tokenizer_save_path)

('./saved_model/tokenizer/tokenizer_config.json',
 './saved_model/tokenizer/special_tokens_map.json',
 './saved_model/tokenizer/vocab.txt',
 './saved_model/tokenizer/added_tokens.json')

## Rotulação dos dados restantes

Agora que temos um modelo já salvo, utilizamos o modelo e o Tokenizer para rotular o restante dos dados do dataset, que não estavam rotulados. Como o modelo foi refinado anteriormente, agora utilizamos o modelo para rotular os dados e salvar em um arquivo parquet, para tentar preservar sua integridade.

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_save_path = './saved_model/model'
tokenizer_save_path = './saved_model/tokenizer'
tokenizer = BertTokenizer.from_pretrained(tokenizer_save_path)
model = BertForTokenClassification.from_pretrained(model_save_path).to(device)

id2label = model.config.id2label

def label_text(text):
    encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).squeeze().cpu().tolist()
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze().cpu().tolist())
    labels = [id2label[pred] for pred in predictions]
    return list(zip(tokens, labels))

news_data = pd.read_parquet('news_data.parquet')

results = []
for index, row in news_data.iterrows():
    labeled_text = label_text(row['content'])
    results.append({'title': row['title'], 'url': row['url'], 'date': row['date'], 'content': labeled_text})

df_results = pd.DataFrame(results)

df_results.to_parquet('labeled_news_data.parquet', index=False)

print("Dataset rotulado e salvo com sucesso!")

Using device: cuda
Dataset rotulado e salvo com sucesso!


## Conversão para outras formas

Por fim, converti os arquivos para ler em outras formas os dados e poder prosseguir com as próximas etapas. O Label-Studio não lê arquivos em parquet, por exemplo, então eu precisava deixar em outros formatos para poder fazer o Pos Annotation por lá.

In [23]:
parquet_file = 'labeled_news_data.parquet'
df = pd.read_parquet(parquet_file)

df.to_json('labeled_news_data.json', orient='records', lines=True, force_ascii=False, indent=4)
print("JSON salvo")

df.to_csv('labeled_news_data.csv', index=False)
print("CSV salvo")

df.to_csv('labeled_news_data.txt', index=False, sep='\t')
print("TXT salvo")


JSON salvo
CSV salvo
TXT salvo
